# FoDS - Assignment 1
## Members:
- Abhinav Sukumar Rao - 2018A7PS0172H  
- Pratyush Banerjee   - 2018A7PS0312H  
- Rohan Daniel        - 2018A7PS0584H  

## Import all packages required

In [1]:
from scipy.special import gamma
from scipy.stats import bernoulli
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

## Generate data

In [4]:
%%capture
import random
p = 0.1
f = p*160
data = []
data = [1 if i<f else 0 for i in range(160)]
data = np.array(data)
random.shuffle(data)
#data = np.array(bernoulli.rvs(size=160,p=0.1))
fig = plt.figure()
x = np.linspace(0,1,200)

In [5]:
print(data)
print(np.count_nonzero(data)/len(data))

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
0.1


In [6]:
%%capture
fig = plt.figure()
ax = plt.axes(xlim=(0,1),ylim=(0,20))
line, = ax.plot([],[])
text = ax.text(0.4,15,"Hi")
def init():
  line.set_data([],[])
  return line,text

## Step 1
Take points one-by-one
### Calculate beta distribution
In order to prevent problems with calculating the beta distribution for large values of a and b using the `scipy.special.gamma` function mentioned in the assignment, we have derived recurrence relations in $a$ and $b$ for the coefficient in the beta distribution:  

$\beta(x,a,b) = \frac{1}{B(a,b)} x^{a-1}(1-x)^{b-1}$    

where,  

$B(a,b) = \frac{\Gamma(a)\Gamma(b)}{\Gamma(a+b)}$  

The recurrence relation(s) are as follows:  

$\beta(a,b) = \frac{(a+b-1)}{(a-1)} \beta(a-1,b)x$

 and,  

$\beta(a,b) = \frac{(a+b-1)}{(b-1)} \beta(a,b-1)(1-x)$

While this is not necessary for the purpose of this assignment, we can use the following function for much larger number of iterations, when we sample one-by-one.

In [7]:
##define a recurrence in gamma to prevent inf errors
## Assumes that a,b are strictly increasing everytime beta is called
def beta(x,a,b):
  if (beta.B is None):
    beta.B = 1/(gamma(a)*gamma(b)/(gamma(a+b)))
    beta.B *= np.power(x,a-1)*np.power(1-x,b-1)
    beta.a = a
    beta.b = b
  elif (a>beta.a):
    beta.a = a
    beta.B = (a+b-1)/(a-1) * beta.B * x
  else:
    beta.b = b
    beta.B = (a+b-1)/(b-1) * beta.B * (1-x)
  return beta.B
beta.a = beta.b = -1
beta.B = None



In [ ]:
'''
def B(a,b):
  res = 1
  if (B.firstrun):
    B.firstrun = False
    B.a = a
    B.b = b
    while (a > 1):
      res *= (a+b-1)/(a-1)
      a-=1
    ##B(a,b) = B(b,a)
    while (b > 1):
      res *= (a+b-1)/(b-1)
      b-=1
   #B(1,1) = 1/2
    B.gamma *= res
  else:
    while (a > B.a):
      res *= (a+b-1)/(a-1)
      a-=1
    while (b > B.b):
      res *= (a+b-1)/(a-1)
      b-=1
    B.gamma*=res
  return B.gamma
B.a = B.b = 0
B.gamma = 1/2
B.firstrun = True
'''

In [8]:
#make an animator function for gif
def an(i):
  global line, data, text
  sample = data[i]
  an.n = an.n+1
  an.m = an.m+sample
  an.a = an.a+sample
  an.b = an.b+(1-sample)
  an.mu = an.a/(an.a+an.b)
  y = beta(x,an.a,an.b)
  #print(type(x),type(y))
  line.set_data(x,y)
  text.set_text(f"mean: {an.mu:.3f}\niteration: {an.n}")
  #print(type(ax),type(line))
  return line,text
  #print(mu,a,b)
an.a = 2
an.b = 3
an.m = 0
an.n = 0
an.mu = 0.4

In [9]:
#reset static vars
def reset():
  an.a = 2
  an.b = 3
  an.m = 0
  an.n = 0
  an.mu = 0.4
  beta.a = beta.b = -1
  beta.B = None
  '''
  B.a = B.b = 0
  B.gamma = 1/2
  B.firstrun = True
  '''
reset()

In [8]:
##gif saver
#%%capture
#!apt install imagemagick

In [10]:
#Write a gif
anim = animation.FuncAnimation(fig, an, init_func=init,
                               frames=160, interval=20, blit=True)
anim.save('./OneByOne.gif',writer='imagemagick')
print(f"160 iterations: mean: {an.mu}")

160 iterations: mean: 0.10909090909090909


## Step 2
Take all the points at once!

In [11]:
def animate_func_2():
  m = np.count_nonzero(data)
  a = 2.0
  b = 3
  mu = a/(a+b)
  n = 160
  y = gamma(a+b)/(gamma(a)*gamma(b))
  y*=np.power(x,a-1)*np.power(1-x,b-1)
  line.set_data(x,y)
  text.set_text(f"mean: {mu:.3f}")
  ax.figure.savefig('./AllAtOncePrior.png')
  a = a+m
  b = b+(n-m)
  mu = a/(a+b)
  y = gamma(a+b)/(gamma(a)*gamma(b))
  y*=np.power(x,a-1)*np.power(1-x,b-1)
  line.set_data(x,y)
  text.set_text(f"mean: {mu:.3f}")
  ax.figure.savefig('./AllAtOncePosterior.png')
  print(f"A single iteration of all 160 points: {mu}")
  return mu

In [12]:
mean_val = animate_func_2()
print(f"Comparing with previous value:{an.mu}")
print(f"OneByOne = AllAtOnce: {an.mu == mean_val}")

A single iteration of all 160 points: 0.10909090909090909
Comparing with previous value:0.10909090909090909
OneByOne = AllAtOnce: True


## Result
We see that the mean of the posterior distribution evaluates to the same result regardless of the method
